# Visualizing NEAR's flyby

The orbital elements are constructed from Horizons-given position and velocity at end of two-way Doppler from DSS 25 (Goldstone). To avoid hitting Horizons needlessly, the trajectory is constructed using `poliastro`'s orbit propagation simply, without involving atmospheric, solar wind, precession, etc., as all such effects are relatively very small.  

In [1]:
from astropy import units as u
from astropy.time import Time, TimeDelta

from poliastro.util import norm, time_range
from poliastro.frames import Planes
from poliastro.ephem import Ephem
from poliastro.bodies import Sun, Earth

from poliastro.twobody.orbit import Orbit
from poliastro.twobody.sampling import EpochsArray

from astropy.coordinates import solar_system_ephemeris

import numpy as np
import sys
sys.path.append('../')

from sim.stations import dss25, dss34, ssrAltair, ssrMillstone
from sim.tracking import Tracking
from sim.util import describe_orbit, describe_state, describe_trajectory

solar_system_ephemeris.set("de440")
start_epoch = Tracking.NEAR_GOLDSTONE_END.value - 1*u.hour
end_epoch = Tracking.NEAR_CANBERRA_START.value + 1*u.hour
sampling_interval = 60

ModuleNotFoundError: No module named 'sim'

In [ ]:
flyby_epochs = start_epoch + (np.arange(0, (end_epoch-start_epoch)/(1*u.s), sampling_interval) << u.s)
print("Start and end epochs:", start_epoch, flyby_epochs[-1], "[", flyby_epochs.size, "]")

In [ ]:
near_start_ephem = Ephem.from_horizons("NEAR", start_epoch, attractor=Earth, plane=Planes.EARTH_EQUATOR)
near_start_rv = near_start_ephem.rv(start_epoch)
describe_state(near_start_rv, dss25, start_epoch)

near_orbit = Orbit.from_vectors(Earth, near_start_rv[0], near_start_rv[1], start_epoch)
describe_orbit(near_orbit)

In [ ]:
near_ephem = near_orbit.to_ephem(EpochsArray(flyby_epochs))
describe_trajectory(near_ephem, dss25)

In [ ]:
from czml3.widget import CZMLWidget
from poliastro.czml.extract_czml import CZMLExtractor

czml = CZMLExtractor(flyby_epochs[0], flyby_epochs[-1], flyby_epochs.size)
#dss25.add_to_czml(czml, "#333")
#dss34.add_to_czml(czml, "#333")
ssrAltair.add_to_czml(czml, "#111")
ssrMillstone.add_to_czml(czml, "#111")
czml.add_orbit(near_orbit)

CZMLWidget(czml.get_document())